# Interface Segregation Principle

The Interface Segregation Principle states that no client should be forced to depend on interfaces they do not use.
This principle emphasizes designing fine-grained, specific interfaces rather than large, general-purpose ones. By doing so, it reduces system complexity and improves code maintainability, as clients interact only with interfaces relevant to their needs.

For example, instead of having a single large interface with several unrelated methods, splitting it into smaller, focused interfaces ensures that implementing classes adhere only to methods they require.

**PaymentProcessor** before segregating into multifactor authentication:

In [ ]:
from abc import ABC, abstractmethod


class Order:
    def __init__(self):
        self.items = []
        self.quantities = []
        self.prices = []
        self.status = "open"

    def add_item(self, name: str, quantity: int, price: float) -> None:
        self.items.append(name)
        self.quantities.append(quantity)
        self.prices.append(price)

    def total_price(self):
        total = 0
        for quantity, price in zip(self.quantities, self.prices):
            total += quantity * price
        return total


class PaymentProcessor(ABC):

    @abstractmethod
    def pay(self, order):
        pass


class DebitCardPaymentProcessor(PaymentProcessor):

    def __init__(self, security_code):
        self.security_code = security_code

    def pay(self, order):
        print("Processing credit card payment")
        print(f"Verifying security code: {self.security_code}")
        order.status = "paid"


class CreditCardPaymentProcessor(PaymentProcessor):

    def __init__(self, security_code):
        self.security_code = security_code

    def pay(self, order):
        print("Processing credit card payment")
        print(f"Verifying security code: {self.security_code}")
        order.status = "paid"


class PaypalPaymentProcessor(PaymentProcessor):

    def __init__(self, email_address):
        self.email_address = email_address

    def pay(self, order):
        print("Processing paypal payment")
        print(f"Verifying Email Address: {self.email_address}")
        order.status = "paid"


order = Order()

order.add_item("Keyboard", 1, 50)
order.add_item("SSD", 1, 150)
order.add_item("USB cable", 2, 5)

processor = CreditCardPaymentProcessor("0372846")
processor.pay(order)
print(order.total_price(), order.status)
order.add_item("Mouse", 1, 100)
paypal_processor = PaypalPaymentProcessor("mohit.gureja@paypal.com")
paypal_processor.pay(order)
print(order.total_price(), order.status)

This principle states that clients should not be forced to depend on methods they do not use. This means it’s better to have interfaces that are suited to specific task rather than one general-purpose interface.

I’ll give an example where we add the ability to send the user an SMS to authenticate their payment.

In [2]:
from abc import ABC, abstractmethod


class Order:
    def __init__(self):
        self.items = []
        self.quantities = []
        self.prices = []
        self.status = "open"

    def add_item(self, name: str, quantity: int, price: float) -> None:
        self.items.append(name)
        self.quantities.append(quantity)
        self.prices.append(price)

    def total_price(self):
        total = 0
        for quantity, price in zip(self.quantities, self.prices):
            total += quantity * price
        return total


class PaymentProcessor(ABC):

    @abstractmethod
    def pay(self, order):
        pass

    @abstractmethod
    def auth_sms(self, code):
        pass


class DebitCardPaymentProcessor(PaymentProcessor):

    def __init__(self, security_code):
        self.security_code = security_code
        self.authenticated = False

    def auth_sms(self, order, code):
        print("Authorizing...")
        self.authenticated = True

    def pay(self, order):
        print("Processing credit card payment")
        print(f"Verifying security code: {self.security_code}")
        if self.authenticated:
            order.status = "paid"
        else:
            raise ValueError("Not authenticated!")


class CreditCardPaymentProcessor(PaymentProcessor):

    def __init__(self, security_code):
        self.security_code = security_code

    def pay(self, order):
        print("Processing credit card payment")
        print(f"Verifying security code: {self.security_code}")
        order.status = "paid"


class PaypalPaymentProcessor(PaymentProcessor):

    def __init__(self, email_address):
        self.email_address = email_address

    def pay(self, order):
        print("Processing paypal payment")
        print(f"Verifying Email Address: {self.email_address}")
        order.status = "paid"


order = Order()

order.add_item("Keyboard", 1, 50)
order.add_item("SSD", 1, 150)
order.add_item("USB cable", 2, 5)

processor = DebitCardPaymentProcessor("0372846")
processor.auth_sms(order, "7604")
processor.pay(order)
print(order.total_price(), order.status)
order.add_item("Mouse", 1, 100)
paypal_processor = PaypalPaymentProcessor("mohit.gureja@paypal.com")
paypal_processor.pay(order)
print(order.total_price(), order.status)

Authorizing...
Processing credit card payment
Verifying security code: 0372846
210 paid


TypeError: Can't instantiate abstract class PaypalPaymentProcessor with abstract method auth_sms

This code violates the ISP because the *CreditPaymentProcessor* and *PaypalPaymentProcessor* class is forced to implement the auth_sms method, even though it does not use it. This not only means we end up writing more code, but it could potentially cause bugs if we forget to implement the method.

Let’s refactor this code to adhere to the ISP.

In [4]:
class PaymentProcessor(ABC):

    @abstractmethod
    def pay(self, order):
        pass


class SMSPaymentProcessor(PaymentProcessor):

    @abstractmethod
    def auth_sms(self, order: Order, code: str):
        pass


class DebitCardPaymentProcessor(SMSPaymentProcessor):

    def __init__(self, security_code):
        self.security_code = security_code
        self.authenticated = False

    def auth_sms(self, order, code):
        print("Authorizing...")
        self.authenticated = True

    def pay(self, order):
        print("Processing credit card payment")
        print(f"Verifying security code: {self.security_code}")
        if self.authenticated:
            print("Order is already authorized")
            order.status = "paid"
        else:
            raise ValueError("Not authenticated!")


class CreditCardPaymentProcessor(PaymentProcessor):

    def __init__(self, security_code):
        self.security_code = security_code

    def pay(self, order):
        print("Processing credit card payment")
        print(f"Verifying security code: {self.security_code}")
        order.status = "paid"


class PaypalPaymentProcessor(PaymentProcessor):

    def __init__(self, email_address):
        self.email_address = email_address

    def pay(self, order):
        print("Processing paypal payment")
        print(f"Verifying Email Address: {self.email_address}")
        order.status = "paid"


order = Order()

order.add_item("Keyboard", 1, 50)
order.add_item("SSD", 1, 150)
order.add_item("USB cable", 2, 5)

processor = DebitCardPaymentProcessor("0372846")
processor.auth_sms(order, "7604")
processor.pay(order)
print(order.total_price(), order.status)
order.add_item("Mouse", 1, 100)
paypal_processor = PaypalPaymentProcessor("mohit.gureja@paypal.com")
paypal_processor.pay(order)
print(order.total_price(), order.status)

Authorizing...
Processing credit card payment
Verifying security code: 0372846
Order is already authorized
210 paid
Processing paypal payment
Verifying Email Address: mohit.gureja@paypal.com
310 paid


Now, we could make this code even better by separating the authorization logic from the payment processor.

We can use composition logic for authorization over extra subclasses, depending on the behaviour we need

In [9]:
class PaymentProcessor(ABC):

    @abstractmethod
    def pay(self, order):
        pass


class SMSAuthorizer:
    authorized = False

    def verify(self, code):
        print(f"Verifying code: {code}")
        self.authorized = True

    def is_authorized(self):
        return self.authorized


class DebitCardPaymentProcessor(PaymentProcessor):

    def __init__(self, security_code, authorizer: SMSAuthorizer):
        self.security_code = security_code
        self.authorizer = authorizer

    def pay(self, order: Order):
        print("Processing debit card payment")
        print(f"Verifying security code: {self.security_code}")
        if self.authorizer.is_authorized():
            print("Order is already authorized")
            order.status = "paid"
        else:
            raise ValueError("Not authenticated!")


class CreditCardPaymentProcessor(PaymentProcessor):

    def __init__(self, security_code):
        self.security_code = security_code

    def pay(self, order: Order):
        print("Processing credit card payment")
        print(f"Verifying security code: {self.security_code}")
        order.status = "paid"


class PaypalPaymentProcessor(PaymentProcessor):

    def __init__(self, email_address, authorizer: SMSAuthorizer):
        self.email_address = email_address
        self.authorizer = authorizer

    def pay(self, order: Order):
        print("Processing paypal payment")
        print(f"Verifying Email Address: {self.email_address}")
        if self.authorizer.is_authorized():
            print("Order is authorized")
            order.status = "paid"
        else:
            raise ValueError("Not authenticated!")

order = Order()
order.add_item("Keyboard", 1, 50)
order.add_item("SSD", 1, 150)
order.add_item("USB cable", 2, 5)

authorizer = SMSAuthorizer()
authorizer.verify("7604")

processor = DebitCardPaymentProcessor("0372846", authorizer)
processor.pay(order)
print(order.total_price(), order.status)


paypal_authorizer = SMSAuthorizer()
paypal_authorizer.verify(5679)

order.add_item("Mouse", 1, 100)
paypal_processor = PaypalPaymentProcessor("mohit.gureja@paypal.com", paypal_authorizer)
paypal_processor.pay(order)
print(order.total_price(), order.status)

Verifying code: 7604
Processing debit card payment
Verifying security code: 0372846
Order is already authorized
210 paid
Verifying code: 5679
Processing paypal payment
Verifying Email Address: mohit.gureja@paypal.com
Order is authorized
310 paid
